In [191]:
import re
import pandas as pd
import geonamescache
from unidecode import unidecode

gc = geonamescache.GeonamesCache()
cities = gc.get_cities()
# print(cities)

black_lst = ["Can", "Come", "Of"]
city_dict_regexp = {}
city_lst = []
city_lst_regexp = []
city_str = ""
df_data = []

items = cities.items()
for item in items:
    city = item[1]['name']
    countrycode = item[1]['countrycode']
    latitude = item[1]['latitude']
    longitude = item[1]['longitude']
    if city not in black_lst:
        city = unidecode(city)
        city_dict_regexp[city] = [countrycode, latitude, longitude]

city_lst_regexp = [*city_dict_regexp]
city_lst_regexp.sort(key = len)  
city_lst_regexp.reverse()

city_dict_regexp = {k.lower(): v for k, v in city_dict_regexp.items()}

city_str = '|'.join(city_lst_regexp)
city_str = "\\b(" + city_str + ")\\b"

regexp = re.compile(r"(?P<city>{cities})".format(cities=city_str), re.I)


In [192]:
count = 0
cnt_ok = 0
cnt_nok = 0

country_dict = gc.get_countries()

with open("data/headlines.txt", "r") as fn:
    lines = fn.readlines()
    for line in lines:
        count = count + 1
        line = line.strip()
        result = regexp.search(line)
        if result != None:
            cnt_ok += 1
            city = result.group('city')
            city_hlp = city.lower()
            
            if city_hlp in city_dict_regexp:
                cc = city_dict[city_hlp][0]
                country = country_dict[cc]['name']
                latitude = city_dict_regexp[city_hlp][1]
                longitude = city_dict_regexp[city_hlp][2]
                # print(cnt_ok, "--", line, "--", city, "--", latitude, "--", longitude, "--", cc, "--", country)
                df_data.append((line, city, latitude, longitude, cc, country))
   
            else:
                None
                # print(count, "DBG-002:", "--", city, "not found in dictionary")
        else:
            cnt_nok +=1
            # print(count, "DBG-001:", "--", line, "not found in headlines")


1 -- Zika Outbreak Hits Miami -- Miami -- 25.77427 -- -80.19366 -- US -- United States
2 -- Could Zika Reach New York City? -- New York City -- 40.71427 -- -74.00597 -- US -- United States
3 -- First Case of Zika in Miami Beach -- Miami Beach -- 25.79065 -- -80.13005 -- US -- United States
4 -- Mystery Virus Spreads in Recife, Brazil -- Recife -- -8.05389 -- -34.88111 -- BR -- Brazil
5 -- Dallas man comes down with case of Zika -- Dallas -- 44.91928 -- -123.31705 -- US -- United States
6 -- Trinidad confirms first Zika case -- Trinidad -- -33.5165 -- -56.89957 -- UY -- Uruguay
7 -- Zika Concerns are Spreading in Houston -- Houston -- 29.76328 -- -95.36327 -- US -- United States
8 -- Geneve Scientists Battle to Find Cure -- Geneve -- 46.20222 -- 6.14569 -- CH -- Switzerland
9 -- The CDC in Atlanta is Growing Worried -- Atlanta -- 33.749 -- -84.38798 -- US -- United States
10 -- Zika Infested Monkeys in Sao Paulo -- Sao Paulo -- -23.5475 -- -46.63611 -- BR -- Brazil
11 -- Brownsville tee

109 -- Is Zika virus in Majuro? -- Majuro -- 7.08971 -- 171.38027 -- MH -- Marshall Islands
110 -- Spreading Zika ruining Cancun fun -- Cancun -- 21.17429 -- -86.84656 -- MX -- Mexico
111 -- Cancun hit by Outbreak of Party Fever! -- Cancun -- 21.17429 -- -86.84656 -- MX -- Mexico
112 -- HIV has Arrived in Fairfax -- Fairfax -- 38.84622 -- -77.30637 -- US -- United States
113 -- Spanish Flu Outbreak in Barcelona -- Barcelona -- 10.13625 -- -64.68618 -- VE -- Venezuela
114 -- Santa Barbara tests new cure for Hepatitis C -- Santa Barbara -- 34.42083 -- -119.69819 -- US -- United States
115 -- Spring break ruined by Zika in Fort Lauderdale -- Spring -- 30.07994 -- -95.41716 -- US -- United States
116 -- Villavicencio under Zika threat -- Villavicencio -- 4.142 -- -73.62664 -- CO -- Colombia
117 -- Two more Zika cases confirmed in Laredo -- Laredo -- 27.50641 -- -99.50754 -- US -- United States
118 -- New Delhi Addressing Zika Concerns -- New Delhi -- 28.63576 -- 77.22445 -- IN -- India
119

216 -- Authorities are Worried about the Spread of Rotavirus in Union -- Union -- 40.6976 -- -74.2632 -- US -- United States
217 -- Rumors about Hepatitis D spreading in Albany have been refuted -- Albany -- 44.63651 -- -123.10593 -- US -- United States
218 -- Zika symptoms spotted in Bello -- Bello -- 6.33732 -- -75.55795 -- CO -- Colombia
219 -- Will Meningitis vaccine help Hamburg? -- Hamburg -- 53.57532 -- 10.01534 -- DE -- Germany
220 -- Madera Patient in Critical Condition after Contracting Rabies -- Madera -- 36.96134 -- -120.06072 -- US -- United States
221 -- Zika Virus Detected in Lubbock -- Lubbock -- 33.57786 -- -101.85517 -- US -- United States
222 -- Malaria re-emerges in Boise -- Boise -- 43.6135 -- -116.20345 -- US -- United States
223 -- Varicella re-emerges in Lagos -- Lagos -- 37.10202 -- -8.67422 -- PT -- Portugal
224 -- Vericella spreading in Ibadan -- Ibadan -- 7.37756 -- 3.90591 -- NG -- Nigeria
225 -- Schools in Birmingham Closed Due to Measles Outbreak -- Birmi

311 -- West Nile Virus Symptoms Spread all over Yerevan -- Yerevan -- 40.18111 -- 44.51361 -- AM -- Armenia
312 -- Zika Outbreak in Sincelejo -- Sincelejo -- 9.30472 -- -75.39778 -- CO -- Colombia
313 -- Rumors about HIV spreading in Montpellier have been refuted -- Montpellier -- 43.61092 -- 3.87723 -- FR -- France
314 -- Praia tests new cure for Pneumonia -- Praia -- 14.93152 -- -23.51254 -- CV -- Cabo Verde
315 -- Lower Hospitalization in Lakewood after Hepatitis B Vaccine becomes Mandatory -- Lakewood -- 47.17176 -- -122.51846 -- US -- United States
316 -- Zika spreads to Kuching -- Kuching -- 1.55 -- 110.33333 -- MY -- Malaysia
317 -- Spike of Rotavirus Cases in Omaha -- Omaha -- 41.25626 -- -95.94043 -- US -- United States
318 -- Tuberculosis re-emerges in Silver Springs -- Springs -- -26.25 -- 28.4 -- ZA -- South Africa
319 -- Will West Nile Virus vaccine help Coronado? -- Coronado -- 32.68589 -- -117.18309 -- US -- United States
320 -- Zika case confirmed in Lorain -- Lorain --

414 -- Biloxi Residents Recieve Chlamydia vaccine -- Biloxi -- 30.39603 -- -88.88531 -- US -- United States
415 -- Case of Mad Cow Disease Reported in Hilden -- Hilden -- 51.16818 -- 6.93093 -- DE -- Germany
416 -- Rumors about Rabies Spreading in Khartoum have been Refuted -- Khartoum -- 15.55177 -- 32.53241 -- SD -- Sudan
417 -- Harare is infested with Pneumonia -- Harare -- -17.82772 -- 31.05337 -- ZW -- Zimbabwe
418 -- Zika symptoms spotted in Sibu -- Sibu -- 2.3 -- 111.81667 -- MY -- Malaysia
419 -- Zika arrives in Quezon -- Quezon -- 7.73028 -- 125.09889 -- PH -- Philippines
420 -- More Patients in Crawfordsville are Getting Diagnosed with Rabies -- Crawfordsville -- 40.04115 -- -86.87445 -- US -- United States
421 -- Spike of Hepatitis B Cases in Greenville -- Greenville -- 33.13845 -- -96.11081 -- US -- United States
422 -- Authorities are Worried about the Spread of Hepatitis D in Akron -- Akron -- 41.08144 -- -81.51901 -- US -- United States
423 -- More Zika patients reported

514 -- More people in Cranford are infected with Hepatitis A every year -- Cranford -- 40.65844 -- -74.29959 -- US -- United States
515 -- West Nile Virus Hits Monroe -- Monroe -- 47.85538 -- -121.97096 -- US -- United States
516 -- Pneumonia Outbreak in Springville -- Springville -- 40.16523 -- -111.61075 -- US -- United States
517 -- Zika arrives in San Onofre -- San Onofre -- 9.73586 -- -75.52626 -- CO -- Colombia
518 -- Lung Cancer Rates Increasing in Moscow -- Moscow -- 46.73239 -- -117.00017 -- US -- United States
519 -- Rumors about Rotavirus Spreading in Joliet have been Refuted -- Joliet -- 41.52519 -- -88.0834 -- US -- United States
520 -- More Zika patients reported in Chanthaburi -- Chanthaburi -- 12.60961 -- 102.10447 -- TH -- Thailand
521 -- Lawrenceville is infested with Dengue -- Lawrenceville -- 33.95621 -- -83.98796 -- US -- United States
522 -- Zika Reported in Ciudad Acuna -- Ciudad Acuna -- 29.32322 -- -100.95217 -- MX -- Mexico
523 -- Zika case reported in Limoeir

612 -- More Zika patients reported in Bella Vista -- Bella Vista -- 36.4807 -- -94.27134 -- US -- United States
613 -- Zika Outbreak in Wichita Falls -- Wichita Falls -- 33.91371 -- -98.49339 -- US -- United States


In [193]:
df = pd.DataFrame(df_data, columns =['headline', 'city', 'latitude', 'longitude', 'countrycode', 'country'])
print("Nr of entries in data frame", df.shape[0])
df.to_csv("output.csv", index=False, header=True)
print(df.iloc[:10])

Nr of entries in data frame 613
                                  headline           city  latitude  \
0                 Zika Outbreak Hits Miami          Miami  25.77427   
1          Could Zika Reach New York City?  New York City  40.71427   
2        First Case of Zika in Miami Beach    Miami Beach  25.79065   
3  Mystery Virus Spreads in Recife, Brazil         Recife  -8.05389   
4  Dallas man comes down with case of Zika         Dallas  44.91928   
5        Trinidad confirms first Zika case       Trinidad -33.51650   
6   Zika Concerns are Spreading in Houston        Houston  29.76328   
7    Geneve Scientists Battle to Find Cure         Geneve  46.20222   
8    The CDC in Atlanta is Growing Worried        Atlanta  33.74900   
9       Zika Infested Monkeys in Sao Paulo      Sao Paulo -23.54750   

   longitude countrycode        country  
0  -80.19366          US  United States  
1  -74.00597          US  United States  
2  -80.13005          US  United States  
3  -34.88111      

In [194]:
print("Total nr of headlines:",count)   
print("Nr OK",cnt_ok)   
print("Nr NOK",cnt_nok)   

Total nr of headlines: 650
Nr OK 613
Nr NOK 37


In [195]:
test = gc.get_cities_by_name('San Bernardino')
test2 = gc.get_cities_by_name('Ho Chi Minh City')

In [196]:
print(test)

[{'5391710': {'geonameid': 5391710, 'name': 'San Bernardino', 'latitude': 34.10834, 'longitude': -117.28977, 'countrycode': 'US', 'population': 216108, 'timezone': 'America/Los_Angeles', 'admin1code': 'CA'}}]


In [197]:
print(test2)

[{'1566083': {'geonameid': 1566083, 'name': 'Ho Chi Minh City', 'latitude': 10.82302, 'longitude': 106.62965, 'countrycode': 'VN', 'population': 3467331, 'timezone': 'Asia/Ho_Chi_Minh', 'admin1code': '20'}}]


In [198]:
test3 = gc.get_cities_by_name('Davos')
print(test3)

[]
